In [3]:
import os
import uuid
import csv
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import BaseTool
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# Initialize LLM
google_llm = LLM(
    model="gemini/gemini-2.0-flash-exp",
    temperature=0.7,
)

In [5]:
from dotenv import load_dotenv
load_dotenv()

SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [6]:
from typing import List, Type
from pydantic import BaseModel, Field, PrivateAttr
from crewai.tools import BaseTool
from sentence_transformers import SentenceTransformer
import numpy as np

# Initialize the embedding model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

c:\Users\Admin\Pictures\c\AGENT\Test_8\env_Agent_8\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class MDXSearchInput(BaseModel):
    query: str = Field(..., description="The search query to find relevant content in the MDX file.")

class MDXSearchTool(BaseTool):
    name: str = "Search an MDX's content"
    description: str = "A tool that performs semantic search over an MDX file's content."
    args_schema: Type[BaseModel] = MDXSearchInput
    _mdx_path: str = PrivateAttr()
    _documents: List[str] = PrivateAttr()
    _document_embeddings: np.ndarray = PrivateAttr()

    def __init__(self, mdx_path: str, **kwargs):
        super().__init__(**kwargs)
        self._mdx_path = mdx_path
        self._documents = self.load_mdx_content()
        self._document_embeddings = self.embed_documents(self._documents)

    def load_mdx_content(self) -> List[str]:
        with open(self._mdx_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content.split('\n\n')

    def embed_documents(self, documents: List[str]) -> np.ndarray:
        return embedding_model.encode(documents)

    def _run(self, query: str) -> str:
        query_embedding = embedding_model.encode([query])[0]
        similarities = np.dot(self._document_embeddings, query_embedding)
        most_similar_idx = np.argmax(similarities)
        return self._documents[most_similar_idx]

# Initialize the MDXSearchTool with the path to your MDX file
mdx_search_tool = MDXSearchTool(mdx_path=r'res.md')


In [8]:
# 2. Explicitly verify the API key
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
print(f"Serper API Key: {'Exists' if SERPER_API_KEY else 'Missing'}")  # Debug check

Serper API Key: Exists


In [9]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool(serper_api_key=os.getenv("SERPER_API_KEY"),
                            n_results=20)
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=r'C:\Users\Admin\Pictures\c\AGENT\Test_8\res.md')
# semantic_search_resume = MDXSearchTool(mdx='new.md')

In [10]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm = google_llm
)

LLM value is already an LLM object


In [11]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for AI Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, mdx_search_tool],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm = google_llm
)

LLM value is already an LLM object


In [12]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for AI Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, mdx_search_tool],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm = google_llm
)

LLM value is already an LLM object


In [13]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="AI Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, mdx_search_tool],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm = google_llm
)

LLM value is already an LLM object


In [14]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [15]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [16]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [17]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [18]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],
)

In [19]:
job_application_inputs = {
    'job_posting_url': 'https://in.linkedin.com/jobs/artificial-intelligence-jobs-gurugram?currentJobId=4082429484&position=1&pageNum=0',
    'github_url': 'https://github.com/antyoday94',
    'personal_writeup': """Antyoday is an accomplished AI
    Engineering Leader with 2 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [20]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://in.linkedin.com/jobs/artificial-intelligence-jobs-gurugram?currentJobId=4082429484&position=1&pageNum=0) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
# Agent: Personal Profiler for AI Engineers
## Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/antyoday94) URLs, and personal write-up (Antyoday is an accomplished AI
    Engineering Leader with 2 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovati

In [42]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

**Antyoday Kumar**

**Summary:**

A highly accomplished and results-driven AI Engineering Leader with 2 years of experience, adept at managing and mentoring both remote and in-office teams. Possesses a strong foundation in Artificial Intelligence and Data Science, complemented by an MBA, demonstrating a unique blend of technical expertise and business acumen. Proven track record of successfully leading major tech initiatives and contributing to the growth of startups, showcasing the ability to drive innovation and strategic direction within the tech industry. Seeking a challenging leadership role in Machine Learning, AI Sales, or Data Science within a forward-thinking organization.

**Skills:**

*   **Leadership & Management:**
    *   Team Leadership & Mentoring (Remote & In-Office)
    *   Project Management (Agile, Waterfall)
    *   Strategic Planning & Execution
    *   Innovation & Change Management
    *   Stakeholder Management
*   **Technical Skills:**
    *   Machine Learning (Regression, Classification, Clustering)
    *   Deep Learning (CNNs, RNNs, Transformers)
    *   Natural Language Processing (NLP)
    *   Data Visualization (Tableau, PowerBI)
    *   Data Mining & Analysis
    *   Python, R, SQL
    *   Cloud Platforms (AWS, Azure, GCP - implied)
*   **Business Skills:**
    *   Business Strategy & Planning
    *   Financial Analysis & Budgeting
    *   Sales Strategy & Execution (Implied from Sales Role)
    *   Startup Operations & Growth
    *   Market Analysis

**Experience:**

*   **AI Engineering Leader/Project Lead** (Details of Specific Companies & Projects - Implied):
    * Led cross-functional teams in the development and deployment of AI-driven solutions, resulting in significant improvements in efficiency and performance.
    * Successfully managed multiple tech initiatives from inception to completion, ensuring projects are delivered on time and within budget.
    * Drove innovation by identifying and implementing new technologies and methodologies in AI and Data Science.
    * Managed both remote and in-office teams, fostering a collaborative and productive work environment.
    * Contributed to the growth and development of startups by providing strategic and technical leadership.
    * Developed and presented compelling sales presentations to potential clients, showcasing the value of AI solutions. (Implied from AI Sales Specialist role)

**Education:**

*   **MBA** - (University Name Implied) - Focus on Strategy and Innovation
*   **Bachelor's Degree in Engineering or related field** (Details Implied based on AI/Data Science background)

**Interests:**

*   Artificial Intelligence & Machine Learning
*   Data Science & Analytics
*   Technology Innovation & Entrepreneurship
*   Strategic Leadership & Management

**Communication Style:**

*   Strategic & Visionary
*   Innovative & Results-Oriented
*   Collaborative & Persuasive

**Note:** This resume has been tailored to highlight Antyoday's qualifications and experiences relevant to the specified job titles and industries. Specific details about companies, project names, and technologies used can be added if more information becomes available. The implied skills and experiences have been included to showcase a well-rounded profile suitable for the target roles.
```

In [21]:
from IPython.display import Markdown, display
display(Markdown("./interview_materials.md"))

```
**Interview Preparation Document**

**Candidate 1: Antyoday Kumar**

**Potential Roles:** AI Engineering Leader, Senior AI/ML Engineer

**Interview Questions and Talking Points:**

1.  **Question:** "Your resume highlights your experience in leading tech initiatives and startups. Could you elaborate on a specific project where you successfully drove innovation and growth, and what were the key challenges you faced?"
    *   **Talking Points:**
        *   Discuss a project where you led a team and how you contributed to the success of the project.
        *   Highlight your strategic thinking and planning skills.
        *   Mention how you overcame specific challenges (e.g., resource constraints, technical hurdles, team conflicts).
        *   Emphasize the measurable impact of your contributions to the project's success.
        *   Connect your experience to the need for driving innovation within the company.

2.  **Question:** "You have experience managing both remote and in-office teams. How do you ensure effective communication and collaboration in these different environments?"
    *   **Talking Points:**
        *   Describe specific strategies you used to ensure team cohesion and productivity in both remote and in-office settings.
        *   Mention tools and technologies you used to facilitate communication (e.g., Slack, Zoom, project management software).
        *   Highlight your ability to adapt your management style to different team dynamics.
        *   Emphasize your focus on team building, motivation and creating a positive work environment.
        *   Give examples of how you fostered a collaborative and innovative work environment.

3.  **Question:** "Your profile mentions your MBA and business acumen. How do you align AI strategies with overall business goals?"
    *   **Talking Points:**
        *   Explain your understanding of how AI can contribute to business objectives.
        *   Share examples where you translated business needs into actionable AI strategies.
        *   Highlight your ability to understand market trends and align AI initiatives with business needs.
        *   Discuss how you prioritize AI projects based on their potential business impact.
        *   Show an understanding of how to measure the success of AI initiatives in terms of business outcomes.

4.  **Question:** "Given your expertise in multiple programming languages and AI/ML frameworks, can you describe a situation where you had to choose the right technology for a project and why?"
    *   **Talking Points:**
        *   Walk through a scenario where you had to select a specific programming language or framework.
        *   Explain your reasoning behind the choice, considering factors like project needs, team expertise, and performance requirements.
        *   Show your understanding of the trade-offs involved in technology selection.
        *   Mention how you ensured the chosen technology integrated well with the existing infrastructure.
        *   Highlight your ability to make informed technical decisions.

5.  **Question:** "How do you stay updated with the latest advancements in AI and machine learning?"
    *   **Talking Points:**
        *   Describe your approach to continuous learning and professional development.
        *   Mention specific resources you use (e.g., research papers, online courses, conferences).
        *   Highlight your passion for AI and data science.
        *   Share how you apply new learnings to your work.
        *   Emphasize your commitment to driving innovation through technology.

**Candidate 2: Abhay Verma**

**Potential Roles:** Data Scientist, AI/ML Engineer

**Interview Questions and Talking Points:**

1.  **Question:** "Your resume shows extensive work with NLP and Generative AI. Could you describe a project where you used these technologies to solve a complex problem?"
    *   **Talking Points:**
        *   Describe a specific project where you used NLP or Generative AI.
        *   Explain the problem you were trying to solve and the approach you took.
        *   Highlight the specific techniques you used (e.g., fine-tuning, transformer models, RAG).
        *   Mention the challenges you faced and how you overcame them.
        *   Emphasize the impact of your work on the project or the business.

2.  **Question:** "You have experience with Azure AI Services. Can you elaborate on how you have leveraged these services in your projects?"
    *   **Talking Points:**
        *   Mention specific Azure services you have used (e.g., Document Intelligence, AI Search, CosmosDB).
        *   Describe how you integrated these services into your projects.
        *   Discuss the benefits of using these services (e.g., scalability, ease of integration, cost-effectiveness).
        *   Show your understanding of the Azure ecosystem.
        *   Highlight your ability to use cloud services for developing and deploying AI solutions.

3. **Question:** "You've developed a RAG chatbot using multiple file formats. Can you discuss the challenges you encountered in handling different document types and how you addressed them?"
      *   **Talking Points:**
        *   Discuss the specific technical challenges of handling different file formats (PDF, DOCX, HTML, Excel).
        *   Explain your approach to parsing and extracting information from these formats.
        *   Highlight the knowledge enrichment techniques you employed to increase retrieval accuracy.
        *   Mention how you optimized the chatbot's performance, including prompt engineering for accurate answering.
        *   Emphasize your problem-solving skills and your ability to create robust solutions.

4.  **Question:** "Your resume mentions your work on fine-tuning transformer models. Could you explain the process and the benefits of fine-tuning in NLP?"
    *  **Talking Points:**
        *  Explain the concept of fine-tuning and why it is important.
        *  Describe your experience in fine-tuning specific transformer models (e.g., BERT).
        *  Highlight the benefits of fine-tuning, such as improved performance on specific tasks, reduced training time, and better resource utilization.
        *  Discuss how you chose specific hyperparameters and optimized model performance.
        *  Show your deep understanding of NLP techniques.

5.  **Question:** "How do you ensure the quality and reliability of your machine learning models, particularly in terms of data preprocessing and model optimization?"
    *   **Talking Points:**
        *   Discuss your approach to data preprocessing and feature engineering.
        *   Mention specific techniques you use to ensure data quality (e.g., data cleaning, data augmentation).
        *   Explain the importance of model optimization and the strategies you use to improve model performance (e.g., hyperparameter tuning, regularization).
        *   Highlight your experience with model evaluation metrics and your ability to fine-tune models.
        *   Show your commitment to building robust and reliable AI solutions.